 # ML for Environmental Engineering 
 By: Suheyla Tozan and Steven Gutterman

In [ ]:
import requests
import pandas as pd
import numpy as np
import time

In [ ]:
#extracts meteorological data from 2018 to 2023 for ELABHWC. For 1993 - 2023, replace "start_year" with 1993

start_year = 2018
end_year = 2023

#parameter codes from API, corresponding to meteorological parameters
param_codes = ["61103", "61104", "62101", "62201", "64101"]

date_ranges = []

#loops over years to extract API URL. NOTE: the URL provided is not updated. To replicate results,
#create your own AQS API account. However, we've made our collected datasets available for the purposes of replicating our plots.
for year in range(start_year, end_year + 1):
    date_range = f"https://aqs.epa.gov/data/api/sampleData/bySite?email=s.gutterman@columbia.edu&key=indigoosprey26&param={','.join(param_codes)}&bdate={year}0101&edate={year}1231&state=06&county=037&site=1103&duration=1"
    date_ranges.append(date_range)

combined_data = pd.DataFrame()

for index, url in enumerate(date_ranges):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raises an HTTPError for bad requests

        data = response.json()
        data = data.get('Data', [])
        df = pd.DataFrame(data)

        df = df[df['parameter_code'].isin(param_codes)][['date_gmt', 'time_gmt', 'parameter_code', 'sample_measurement']]

        for param in param_codes:
            df_param = df[df['parameter_code'] == param].copy()
            df_param.rename(columns={'sample_measurement': f'sample_measurement_{param}'}, inplace=True)
            df_param.drop('parameter_code', axis=1, inplace=True)

            if combined_data.empty:
                combined_data = df_param
            else:
                combined_data = pd.merge(combined_data, df_param, on=['date_gmt', 'time_gmt'], how='outer')

        print(f"Successfully added data from {url} (File {index + 1} of {len(date_ranges)})")

        # Wait for 10 seconds before processing the next URL
        time.sleep(10)

    except requests.HTTPError as http_err:
        print(f"HTTP error occurred while fetching data from {url}: {http_err}")
    except Exception as err:
        print(f"An error occurred while fetching data from {url}: {err}")

# Save the combined data to an Excel file
combined_data.to_excel("metdata_2018-2023_ELABHWC", index=False)

print("All data has been successfully compiled into the Excel file.")

In [ ]:
#now collecting criteria gas concentration data

start_year = 2018
end_year = 2023

#parameter codes from API, corresponding to criteria gas concentration parameters
param_codes = ["42101", "42602", "42401", "44201"]

date_ranges = []

#loops over years to extract API URL. NOTE: the URL provided is not updated. To replicate results,
#create your own AQS API account. However, we've made our collected datasets available for the purposes of replicating our plots.
for year in range(start_year, end_year + 1):
    date_range = f"https://aqs.epa.gov/data/api/sampleData/bySite?email=s.gutterman@columbia.edu&key=indigoosprey26&param={','.join(param_codes)}&bdate={year}0101&edate={year}1231&state=06&county=037&site=1103&duration=1"
    date_ranges.append(date_range)

combined_data = pd.DataFrame()

for index, url in enumerate(date_ranges):
    try:
        response = requests.get(url)
        response.raise_for_status()

        data = response.json()
        data = data.get('Data', [])
        df = pd.DataFrame(data)

        df = df[df['parameter_code'].isin(param_codes)][['date_gmt', 'time_gmt', 'parameter_code', 'sample_measurement']]

        for param in param_codes:
            df_param = df[df['parameter_code'] == param].copy()
            df_param.rename(columns={'sample_measurement': f'sample_measurement_{param}'}, inplace=True)
            df_param.drop('parameter_code', axis=1, inplace=True)

            if combined_data.empty:
                combined_data = df_param
            else:
                combined_data = pd.merge(combined_data, df_param, on=['date_gmt', 'time_gmt'], how='outer')

        print(f"Successfully added data from {url} (File {index + 1} of {len(date_ranges)})")

        time.sleep(10)

    except requests.HTTPError as http_err:
        print(f"HTTP error occurred while fetching data from {url}: {http_err}")
    except Exception as err:
        print(f"An error occurred while fetching data from {url}: {err}")

combined_data.to_excel("criteriagasdata_2018-2023_ELABHWC.xlsx", index=False)

print("All data has been successfully compiled into the Excel file.")

In [ ]:
#this cell consolidates and cleans up columns, shown is for the meteorological parameters, but replace
#file_path and param_codes to consolidate/clean up criteria gas data

file_path = "metdata_2018-2023_ELABHWC.xlsx"
data = pd.read_excel(file_path)

param_codes = ["61103", "61104", "62101", "62201", "64101"]

for param in param_codes:
    param_columns = [col for col in data.columns if f'sample_measurement_{param}' in col and col != f'sample_measurement_{param}']

    if param_columns:
        data[f'sample_measurement_{param}'] = data[param_columns].max(axis=1, skipna=True)

        data.drop(columns=param_columns, inplace=True)

final_columns = ['date_gmt', 'time_gmt'] + [f'sample_measurement_{param}' for param in param_codes]
data = data[final_columns]

data.to_excel("metdata_2018-2023_ELABHWC.xlsx", index=False)

print("Data has been successfully consolidated into the new Excel file.")

In [ ]:
#this cell removes duplicate column names for "date_gmt" and "time_gmt"
#shown is for criteria gas data, but replace with meteorological data to do both
file_path = 'criteriagasdata_2018-2023_ELABHWC.xlsx'
df = pd.read_excel(file_path)

df_cleaned = df.drop_duplicates(subset=['date_gmt', 'time_gmt'])

df_cleaned.to_excel('criteriagasdata_2018-2023_ELABHWC.xlsx', index=False)

print("Duplicate rows based on the first two columns have been removed.")